<a href="https://colab.research.google.com/github/Glebych2/MIPT_home_work/blob/main/Environment_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -U kaggle_environments

     |████████████████████████████████| 1.9 MB 4.1 MB/s 
     |████████████████████████████████| 62 kB 1.3 MB/s 


In [3]:
import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
import seaborn as sns

from kaggle_environments import make, evaluate

In [15]:
print(
    evaluate(
    "rps", #environment to use - no need to change
    ["rnd_agent.py","rnd_agent.py"], #agents to evaluate
    configuration={"episodeSteps": 2, 'signs':3, 'tieRewardThreshold':1, }, #определяем число шагов как 2, так как у нас два агента. Так же определяем, что для выигрыша нужна одна победа
    num_episodes=50 #делаем 50 прогонов, в каждом по 2 шага.
    )
)

TypeError: ignored

Опишем поведение агента, всегда играющего "камень" - это значение 0

In [4]:
%%writefile rock_agent.py
#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 0

Writing rock_agent.py


Попробуем теперь использовать информацию о прошлых действиях противника. Опишем агента, который производит то же самое действие, что и оппонент на прошлом ходу

In [37]:
%%writefile copy_opponent.py
import random
#Example 
def copy_opponent(observation, configuration):   
    #in case we have information about opponent last move
    if observation.step > 0:
        return observation.lastOpponentAction
        #return 2
    #initial step
    else:
        return random.randrange(0, configuration.signs)
        #return random.randrange(0, 2)

Overwriting copy_opponent.py


In [6]:
%%writefile paper_agent.py
#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 1

Writing paper_agent.py


In [7]:
%%writefile scissors_agent.py
#Example of the simple agent
#0 - rock
#1 - paper
#2 - scissors
def your_agent(observation, configuration):
    return 2

Writing scissors_agent.py


In [8]:
%%writefile submission.py

import random

def nash_equilibrium_agent(observation, configuration):
    return random.randint(0, 2)

Writing submission.py


Воспользуемся функцией evaluate из библиотеки kaggle_environments с помощью которой запустим наших агентов и проведем эксперимент на заданном количестве игр

In [115]:
from copy_opponent import copy_opponent
evaluate(
    "rps", #environment to use - no need to change
    ["submission.py", "copy_opponent.py"], #agents to evaluate
    configuration={"episodeSteps": 100, 'tieRewardThreshold': 2} #number of episodes 
)

[[-5.0, 5.0]]

In [106]:
evaluate(
    "rps", #environment to use - no need to change
    ["scissors_agent.py", "paper"], #agents to evaluate
    configuration={"episodeSteps":100} #number of episodes 
)

[[99.0, -99.0]]